In [12]:
# Task 8: Symbolic Logic Inference + Minimal LangChain Setup

# Install dependencies
!pip install -U langchain langchain-community faiss-cpu

# --- KNOWLEDGE BASE SETUP ---
from langchain_core.documents import Document
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings

In [16]:
# Facts and rules (Prolog style)

# kb_facts = """\
# animal(dog).
# animal(cat).
# animal(eagle).
# animal(sparrow).
# flies(eagle).
# flies(sparrow).
# has_tail(dog).
# has_tail(cat).
# """

# kb_rules = """\
# bird(X) :- animal(X), flies(X).
# mammal(X) :- animal(X), has_tail(X).
# warm_blooded(X) :- mammal(X).
# """

# --- Facts and Rules (Prolog Style) ---
kb_facts = """\
animal(dog).
animal(cat).
animal(eagle).
animal(sparrow).
animal(crocodile).
animal(whale).
flies(eagle).
flies(sparrow).
has_tail(dog).
has_tail(cat).
lays_eggs(crocodile).
"""

kb_rules = """\
bird(X) :- animal(X), flies(X).
mammal(X) :- animal(X), has_tail(X).
warm_blooded(X) :- mammal(X).
reptile(X) :- animal(X), lays_eggs(X).
cold_blooded(X) :- reptile(X).
"""


# LangChain RAG setup (to meet Task 8 requirement)
docs = [
    Document(page_content=kb_facts, metadata={"source": "facts"}),
    Document(page_content=kb_rules, metadata={"source": "rules"})
]

splitter = CharacterTextSplitter(chunk_size=100, chunk_overlap=0)
kb_chunks = splitter.split_documents(docs)

embedding = HuggingFaceEmbeddings()
vectorstore = FAISS.from_documents(kb_chunks, embedding)
retriever = vectorstore.as_retriever()

# --- SYMBOLIC INFERENCE ENGINE (Backward Chaining) ---
from collections import defaultdict

# Parse facts
facts = set()
for line in kb_facts.strip().splitlines():
    pred, arg = line.strip(").").split("(")
    facts.add((pred.strip(), arg.strip()))

# Parse rules
rules = defaultdict(list)
for line in kb_rules.strip().splitlines():
    head, body = line.split(":-")
    head_pred, head_var = head.strip(").").split("(")
    body_preds = []
    for pred in body.strip().split(","):
        pred_name, pred_var = pred.strip(").").split("(")
        body_preds.append((pred_name.strip(), pred_var.strip()))
    rules[(head_pred, head_var)].append(body_preds)

# Backward chaining logic inference
def prove(goal_pred, goal_const):
    if (goal_pred, goal_const) in facts:
        return True
    for (head_pred, head_var), rule_bodies in rules.items():
        if head_pred != goal_pred:
            continue
        for body in rule_bodies:
            if all(prove(pred_name, goal_const) for pred_name, _ in body):
                return True
    return False

# --- TEST QUERIES ---
queries = [
    ("mammal", "cat"),
    ("bird", "sparrow"),
    ("warm_blooded", "cat"),
    ("bird", "cat"),
    ("warm_blooded", "sparrow"),
    ("mammal", "eagle"),
]

print("Symbolic Logic Inference Results:\n")
for pred, const in queries:
    result = prove(pred, const)
    print(f"{pred}({const}) → {result}")


<ipython-input-16-e35f9e787e67>:53: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embedding = HuggingFaceEmbeddings()


Symbolic Logic Inference Results:

mammal(cat) → True
bird(sparrow) → True
warm_blooded(cat) → True
bird(cat) → False
warm_blooded(sparrow) → False
mammal(eagle) → False
